This notebook evaluates Score-CAM with UCF-50 frame using the tf-keras-vis implementation. 

This notebook is based on the example presented in tf-keras-vis (Kubota, Y. (2021). tf-keras-vis (Version 0.8.1) [Computer software]. https://keisen.github.io/tf-keras-vis-docs/)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
import cv2

Load frames and masks

In [ ]:
!cp -R /content/gdrive/MyDrive/masked_images_backup/masks /
!cp -R /content/gdrive/MyDrive/video_frames/ /

In [ ]:
!pip install tf-keras-vis

     |████████████████████████████████| 53 kB 863 kB/s 


In [ ]:
import tf_keras_vis

In [ ]:
model = keras.models.load_model('/content/gdrive/MyDrive/resnet50_v2_transferLearned_model')

In [ ]:
from keras.preprocessing.image import load_img
from keras.applications.resnet_v2 import preprocess_input

In [ ]:
from PIL import Image

In [ ]:
classes_list = ['PizzaTossing', 'PlayingGuitar', 'PushUps', 'SalsaSpin']

In [ ]:
from matplotlib import cm
from tf_keras_vis.scorecam import Scorecam

In [ ]:
def create_scoreCAM(class_name, dir_name, check_preds=False):
  images, masks, file_names, X = prepare_data(class_name, dir_name, check_preds)
  replace2linear = ReplaceToLinear()
  # 0 = PizzaTossing, 1 = PlayingGuiar, 2 = PushUps, 3 = SalsaSpin.
  for index, name in enumerate(classes_list):
    if name==class_name:
      score = CategoricalScore([index])
  start = time.time()
  # Create ScoreCAM object
  scorecam = Scorecam(model)
  
  # Generate heatmap with ScoreCAM
  cam = scorecam(score, X, penultimate_layer=-1)

  duration = time.time()-start

  for index, map in enumerate(cam):
    path = file_names[index].replace('video_frames','scoreCAM')

    # Render
    f, ax = plt.subplots(nrows=1, ncols=1, figsize=(4, 4))
    #heatmap = np.uint8(cm.jet(map)[..., :3] * 255)
    ax.set_title(class_name, fontsize=16)
    ax.imshow(images[index])
    ax.imshow(map, cmap='jet', alpha=0.5)
    ax.axis('off')
    plt.tight_layout()
    plt.savefig(path)
    #plt.show()
    plt.close()
  return cam, images, masks, file_names, duration


In [ ]:
def prepare_data(class_name, dir_name, image_path, mask_path, check_preds=False):


  masks_root = '/masks'
  images_root ='/video_frames'


  im = load_img(image_path,target_size=(100,100))
  im = np.array(im)
  x = np.asarray([im])
  x = preprocess_input(x)
  pred = model.predict(x)

  if not os.path.exists('/scoreCAM'):
    os.mkdir('/scoreCAM')
  if not os.path.exists(os.path.join('/scoreCAM',class_name)):
    os.mkdir(os.path.join('/scoreCAM',class_name))
  if not os.path.exists(os.path.join('/scoreCAM', class_name, dir_name)):
    os.mkdir(os.path.join('/scoreCAM', class_name, dir_name))
  
  image = None
  mask = None
  file_name = None
  if classes_list[np.argmax(pred[0])] == class_name or check_preds:

    file_name = image_path



    msk = np.array(Image.open(mask_path).resize(im.shape[1::-1], Image.BILINEAR))
    msk = msk/255

    image= im
    mask=msk
    
  
  image_batch = np.asarray([image])

  # Preparing input data
  X = preprocess_input(image_batch)

  

  return image, mask, file_name, X

In [ ]:
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore
from keras import backend as K
import time

In [ ]:
pizza = [('PizzaTossing','v_PizzaTossing_g02_c02')]
salsa = [('SalsaSpin','v_SalsaSpin_g04_c02'),('SalsaSpin','v_SalsaSpin_g11_c02'),('SalsaSpin','v_SalsaSpin_g18_c04')]
guitar = [('PlayingGuitar', 'v_PlayingGuitar_g06_c05'), ('PlayingGuitar', 'v_PlayingGuitar_g11_c02'),('PlayingGuitar','v_PlayingGuitar_g22_c03')]
pushups = [('PushUps', 'v_PushUps_g11_c03'), ('PushUps','v_PushUps_g14_c04'),('PushUps','v_PushUps_g18_c02')]

#all_classes = [pizza, salsa, guitar, pushups]

In [ ]:
def initiate_eval(class_name, dir_name, check_preds=False):
  masks_root = '/masks'
  images_root ='/video_frames'

  
  
  
  maps = []
  images = []
  masks = []
  file_names = []
  durations = []

  for mask_name in os.listdir(os.path.join(masks_root, class_name, dir_name, 'masks')):
    image_name = mask_name.replace('png','jpg')
    image_path = os.path.join(images_root,class_name,dir_name,image_name)
    mask_path = os.path.join(masks_root, class_name, dir_name, 'masks', mask_name)

    img, msk, file_name, x = prepare_data(class_name, dir_name, image_path, mask_path, check_preds)

    if img is not None:
  

      replace2linear = ReplaceToLinear()

      # 0 = PizzaTossing, 1 = PlayingGuiar, 2 = PushUps, 3 = SalsaSpin.
      for index, name in enumerate(classes_list):
        if name==class_name:
          score = CategoricalScore([index])
      

      start = time.time()
      # Create ScoreCAM object
      scorecam = Scorecam(model)
      
      # Generate heatmap with ScoreCAM
      cam = scorecam(score, x, penultimate_layer=-1)

      duration = time.time()-start

      for index, map in enumerate(cam):
        path = file_name.replace('video_frames','scoreCAM')

        # Render
        f, ax = plt.subplots(nrows=1, ncols=1, figsize=(4, 4))
        #heatmap = np.uint8(cm.jet(map)[..., :3] * 255)
        ax.set_title(class_name, fontsize=16)
        ax.imshow(img)
        ax.imshow(map, cmap='jet', alpha=0.5)
        ax.axis('off')
        plt.tight_layout()
        plt.savefig(path)
        #plt.show()
        plt.close()
        
        maps.append(cam)
        images.append(img)
        masks.append(msk)
        file_names.append(file_name)
        durations.append(duration)

  return maps, images, masks, file_names, durations

In [ ]:
def evaluate_maps(maps, masks, duration):
  duration = duration/ len(maps)
  duration_list = []
  duration_list.extend([duration]*len(maps))

  map_weight_list = []
  mask_weight_list = []
  for index, map in enumerate(maps):
    map_weight = np.sum(map)
    dst = map * masks[index]
    mask_weight = np.sum(dst)

    map_weight_list.append(map_weight)
    mask_weight_list.append(mask_weight)

  return map_weight_list, mask_weight_list, duration_list



In [ ]:
map_weights = []
mask_weights = []
durations = []
all_classes= [pizza, salsa]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , durations = initiate_eval(name, dir)
    duration = sum(durations)/len(durations)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')
if not os.path.exists('/content/gdrive/MyDrive/Xception'):
  os.mkdir('/content/gdrive/MyDrive/Xception')
!cp -R /scoreCAM /content/gdrive/MyDrive/Xception/

map_weights = []
mask_weights = []
durations = []
all_classes= [pizza]

for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , durations = initiate_eval(name, dir)
    duration = sum(durations)/len(durations)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')

map_weights = []
mask_weights = []
durations = []
all_classes= [salsa]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , durations = initiate_eval(name, dir)
    duration = sum(durations)/len(durations)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')


map_weights = []
mask_weights = []
durations = []
all_classes= [pushups]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , durations = initiate_eval(name, dir)
    duration = sum(durations)/len(durations)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration, check_preds=True)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')
if not os.path.exists('/content/gdrive/MyDrive/Xception/PushUps'):
  os.mkdir('/content/gdrive/MyDrive/Xception/PushUps')
!cp -R /scoreCAM /content/gdrive/MyDrive/Xception/PushUps/

map_weights = []
mask_weights = []
durations = []
all_classes= [guitar]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , durations = initiate_eval(name, dir)
    duration = sum(durations)/len(durations)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration, check_preds=True)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')
if not os.path.exists('/content/gdrive/MyDrive/Xception/PlayingGuitar'):
  os.mkdir('/content/gdrive/MyDrive/Xception/PlayingGuitar')
!cp -R /scoreCAM /content/gdrive/MyDrive/Xception/PlayingGuitar/


map_weights = []
mask_weights = []
durations = []
all_classes= [pizza]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , durations = initiate_eval(name, dir)
    duration = sum(durations)/len(durations)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration, check_preds=True)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')
if not os.path.exists('/content/gdrive/MyDrive/Xception/PizzaTossing'):
  os.mkdir('/content/gdrive/MyDrive/Xception/PizzaTossing')
!cp -R /scoreCAM /content/gdrive/MyDrive/Xception/PizzaTossing/


map_weights = []
mask_weights = []
durations = []
all_classes= [salsa]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , durations = initiate_eval(name, dir)
    duration = sum(durations)/len(durations)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration, check_preds=True)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')
if not os.path.exists('/content/gdrive/MyDrive/Xception/SalsaSpin'):
  os.mkdir('/content/gdrive/MyDrive/Xception/SalsaSpin')
!cp -R /scoreCAM /content/gdrive/MyDrive/Xception/SalsaSpin/


map_weights = []
mask_weights = []
durations = []
all_classes= [pizza, salsa, guitar, pushups]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , durations = initiate_eval(name, dir)
    duration = sum(durations)/len(durations)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration, check_preds=True)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')
